In [3]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
import tensorflow_probability as tfp
from keras import layers, initializers, optimizers, losses

from tensorflow.keras.callbacks import Callback
from tqdm.keras import TqdmCallback
from tqdm import tqdm

2025-10-10 17:01:29.976363: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760126489.994692   21381 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760126489.999808   21381 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760126490.014721   21381 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760126490.014747   21381 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760126490.014750   21381 computation_placer.cc:177] computation placer alr

In order for we to not have to import tensorflow every time we want to test something. Development for the models will be made here and then migrated to the proper .py files.


In [ ]:
class FrailtyModelNN(keras.models.Model):

    def __init__(self, parameters, loglikelihood):
        super().__init__()
        self.parameters = {
            "theta": {"link": lambda x : tf.math.exp(x), "link_inv": lambda x : tf.math.log(x), "par_type": "nn", "update_func": None, "shape": None, "initializer": None},
            "tau": {"link": lambda x : tf.math.exp(x), "link_inv": lambda x : tf.math.log(x), "par_type": "nn", "update_func": None, "shape": 5, "initializer": None},
            "alpha": {"link": lambda x : 1/(1+tf.math.exp(-x)), "link_inv": lambda x : tf.math.log(x) - tf.math.log(1-x), "par_type": "independent", "update_func": None, "shape": None, "init": 0.5},
            "gamma": {"link": lambda x : tf.math.exp(x), "link_inv": lambda x : tf.math.log(x), "par_type": "independent", "update_func": None, "shape": None, "init": 1.0},
            "phi1": {"link": lambda x : tf.math.exp(x), "link_inv": lambda x : tf.math.log(x), "par_type": "independent", "update_func": None, "shape": None, "init": 1.0},
            "phi2": {"link": lambda x : tf.identity(x), "link_inv": lambda x : tf.identity(x), "par_type": "independent", "update_func": None, "shape": None, "init": 0.0}
        }
        self.n_acum_step = tf.Variable(0, dtype = tf.int32, trainable = False)

    def clone_architecture(self, model):
        pass
    
    def define_structure(self):

        # Goes through the list of parameters for the model and filter them by their classes:
        # - "nn" will be treated as an output from a given neural network that receives the variables x as input.
        # - "independet" will be treated an an individual tf.Variable, trainable object. It is still trained in tensorflow, but is constant for all subjects
        # - "fixed" will be treated as a non-trainable tf.Variable. Basically just a known constant.
        # - "manual" will be treated as a non-trainable tf.Variable, but its value will be eventually updated manually using user provided functions (useful in cases where closed forms can be obtained)
        # - "dependent" will be treated simply as a deterministic function of other parameters and will be updated after training

        nn_pars = []
        independent_pars = []
        fixed_pars = []
        manual_pars = []
        for parameter in self.parameters:
            par = self.parameters[parameter]
            if(par["par_type"] == "nn"):
                nn_pars.append( parameter )
            elif(par["par_type"] == "independent"):
                independent_pars.append( parameter )
            elif(par["par_type"] == "fixed"):
                fixed_pars.append( parameter )
            elif(par["par_type"] == "manual"):
                manual_pars.append( parameter )
            else:
                raise Exception("Invalid parameter {} type: {}".format(parameter, par["par_type"]))

        # If at least one parameter is to be modeled as a neural network output, define its architecture here
        if( len(nn_pars) > 0 ):
            # Try to implement a function that, given a model, clone its architecture
            # clone_architecture(given_model)

            # The user provided architecture must output an array with the exact same shape as the concatenation of all "nn" parameters.
            # For example, if theta (nn) is a single constant and alpha (nn) is an array with 3 values, the expected output for the neural network
            # component is 4. For that, these parameters must be at most one dimensional arrays. (No matrix, or more complex structured parameters!)
            initializer = initializers.GlorotNormal(seed = 1)
            self.dense1 = keras.layers.Dense(units = 16, activation = "gelu", kernel_initializer = initializer, dtype = tf.float32, name = "dense1")
            self.dense2 = keras.layers.Dense(units = 6, kernel_initializer = initializer, dtype = tf.float32, activation = None, use_bias = False, name = "output")

        # Dictionary with all parameters that are its individual weights
        self.model_variables = {}

        # Include variables that do not depend on the variables x, but are still trained by tensorflow
        for parameter in independent_pars:
            par = self.parameters[parameter]

            # If shape is None, set it to ()
            if(par["shape"] is None):
                par["shape"] = ()

            raw_init = par["link_inv"]( par["init"] )

            self.model_variables[parameter] = self.add_weight(
                name = "raw_" + parameter,
                shape = par["shape"],
                initializer = keras.initializers.Constant( raw_init ),
                trainable = True,
                dtype = tf.float32
            )

        # Include variables that are not trained by tensorflow (known, fixed constants or manual trained variables)
        for parameter in np.concatenate([fixed_pars, manual_pars]):
            par = self.parameters[parameter]
            self.model_variables[parameter] = self.add_weight(
                name = parameter,
                shape = par["shape"],
                initializer = keras.initializers.Constant( par["initializer"] ),
                trainable = False,
                dtype = tf.float32
            )

        # print("init trainable variables\n")
        # print(self.trainable_variables)

        # Organize trainable variables information, so each variable can get mapped to an index in the self.trainable_variables and its gradients
        self.vars_to_index = {}
        # Before we build the model, the only variables that appear in here are the ones corresponding to "independent" parameters
        for i, var in enumerate(self.trainable_variables):
            # From the variable path, get its name (raw_<variable>)
            var_name = var.path.split("/")[-1]
            # Save its corresponding index
            self.vars_to_index[var_name] = i

        print("INCLUDING NN PARAMETERS IN THE LIST...")
        nn_par_index = 0
        # We must also include in this list the indices for "nn" parameters
        for i, parameter in enumerate(nn_pars):
            par = self.parameters[ parameter ]
            if(par["shape"] is None):
                par_shape = 1
            else:
                # The parameter must be at most a 1-dimensional array, whose indices will be saved for future location in the neural network output results
                par_shape = par["shape"]
            
            self.vars_to_index["raw_" + parameter] = tf.constant( np.arange(nn_par_index, nn_par_index+par_shape), dtype = tf.int32 )
            nn_par_index += par_shape
        
        print("VARIABLES MAPPING TO INDEX")
        print(self.vars_to_index)

    def call(self, x_input):
        x = self.dense1(x_input)
        return self.dense2(x)

    def loss_func(self, variables, nn_output, t, delta):
        """
            This is an example of a correctly defined loss function.

            - eta: Corresponds to the output of the neural network with input x - Corresponds to the 
        """
        
        # ACREDITO QUE DE PRA FAZER ESSA AUTOMAÇÃO DE FORMA AUTOMATIZADA, JÁ QUE O USUÁRIO FORNECE A LINK FUNCTION NO INÍCIO
        # UMA IDEIA É APENAS FORÇAR COM QUE loss_func CUSTOMIZADA SEJA INSERIDA ACEITANDO UM PARAMETRO PARAMETERS
        # EM QUE PAREMETERS SERÁ SIMPLESMENTE UM DICIONÁRIO QUE JÁ COSPE OS VALORES DOS PARÂMETROS PRONTINHO
        # MAS DIFICIL, JÁ QUE COMO O TENSORFLOW DEVE PRESERVAR A ESTRUTURA DOS SEUS OBJETOS PARA CALCULAR AS DERIVADAS PODE NÃO SER POSSIVEL SEGMENTAR ANTES
        # TALVEZ VALHA A PENA DEIXAR COMO ESTÁ. FICA MAIS DIFÍCIL DE ENTENDER, MAS MANTEM A FLEXIBILIDADE PARA O USUÁRIO
        raw_theta = tf.gather(nn_output, self.vars_to_index["raw_theta"], axis = 1)
        raw_tau = tf.gather(nn_output, self.vars_to_index["raw_tau"], axis = 1)
        print(variables)
        alpha = 1/(1+tf.math.exp(-variables["raw_alpha"]))
        gamma = tf.math.exp( variables["raw_gamma"] )
        phi1 = tf.math.exp( variables["raw_phi1"] )
        phi2 = tf.identity( variables["raw_phi2"] )

        # log_f0 = tf.math.log(phi1) + (phi1-1)*tf.math.log(y) + phi2 - tf.math.exp(phi2) * tf.math.pow(y, phi1)
        # F0 = 1 - tf.math.exp(-tf.math.pow(y, phi1) * tf.math.exp(phi2))
        # laplace_transform_term = 1 + gamma*theta*F0/(1-alpha)

        # loss_weights = delta*eta + delta*log_f0 + (1-alpha)/(alpha*gamma)*(1 - tf.math.pow(laplace_transform_term, alpha)) + (alpha-1)*delta*tf.math.log(laplace_transform_term)
        # loss_weights_mean = -tf.math.reduce_mean(loss_weights)
        
        # return loss_weights_mean

        return tf.constant( tf.math.reduce_mean(variables) )

    def train_step(self, data):
        """
            Called by each batch in order to evaluate the loglikelihood and accumulate the parameters gradients using training data.
        """
        x, t, delta = data

        self.n_acum_step.assign_add(1)
        with tf.GradientTape() as tape:
            nn_output = self(x, training = True)

            # likelihood_loss = self.loss_func(variables = self.model_variables, nn_output = nn_output, t = t, delta = delta)

        # print("Trainable variables:\n")
        # print(self.trainable_variables)

        gradients = tape.gradient(likelihood_loss, self.trainable_variables)

        # print("GRADIENTS:\n")
        # print(gradients)

        return {"likelihood_loss": likelihood_loss}

    def compile_model(self, optimizer, learning_rate, run_eagerly):
        """
            Defines the configuration for the model, such as batch size, training mode, early stopping.
        """
        # optimizer = optimizers.Adam(learning_rate = learning_rate, gradient_accumulation_steps = None),
        self.optimizer = optimizer
        self.compile(
            run_eagerly = run_eagerly
        )

    def train_model(self, x, t, delta,
                    epochs, shuffle = True,
                    optimizer = optimizers.Adam(learning_rate = 0.001),
                    train_batch_size = None, val_batch_size = None,
                    buffer_size = 4096, gradient_accumulation_steps = None,
                    run_eagerly = True, verbose = 1):
        """
            This is the function that start the training.
        """
        
        # Pass the input variables to tensorflow default types
        x = tf.cast(x, dtype = tf.float32)
        t = tf.cast(t, dtype = tf.float32)
        delta = tf.cast(delta, dtype = tf.float32)
        
        # If input is a vector, transform it into a column
        if(len(x.shape) == 1):
            x = tf.reshape( x, shape = (len(x), 1) )
        if(len(t.shape) == 1):
            t = tf.reshape( t, shape = (len(t), 1) )
        if(len(delta.shape) == 1):
            delta = tf.reshape( delta, shape = (len(delta), 1) )

        # Salva os dados originais
        self.x = x
        self.t = t
        self.delta = delta

        # If no validation step should be taken, training data is the same as validation data
        self.x_train, self.t_train, self.delta_train = self.x, self.t, self.delta
        self.x_val, self.t_val, self.delta_val = self.x, self.t, self.delta
        
        # Declara os callbacks do modelo
        self.callbacks = [ ]
        
        if(verbose >= 1):
            self.callbacks.append( TqdmCallback(verbose = 0, position = 0, leave = True) )
        
        # If batch_size is unspecified, set it to be the training size. Note that decreasing the batch size to smaller values, such as 500 for example, has previously lead the
        # model to converge too early, leading to a lot of time of investigation. When dealing with neural networks in the statistical models context, we recommend to use a single
        # batch in training. Alternatives in the case that the sample is too big might be to consider a "gradient accumulation" approach.
        self.train_batch_size = train_batch_size
        if(self.train_batch_size is None):
            self.train_batch_size = self.x_train.shape[0]

        self.val_batch_size = val_batch_size
        if(self.val_batch_size is None):
            self.val_batch_size = self.x_val.shape[0]
        
        self.gradient_accumulation_steps = gradient_accumulation_steps
        if(self.gradient_accumulation_steps is None):
            # The number of batches until the actual weights update (we ensure that the weights are updated only once per epoch, even though we might have multiple batches)
            self.gradient_accumulation_steps = int(np.ceil( self.x_train.shape[0] / self.train_batch_size ))

        self.compile_model(optimizer = optimizer, learning_rate = 0.001, run_eagerly = run_eagerly)

        # Create the training dataset
        self.buffer_size = buffer_size
        train_dataset = tf.data.Dataset.from_tensor_slices((self.x_train, self.t_train, self.delta_train))
        train_dataset = train_dataset.shuffle(buffer_size = self.buffer_size).batch(self.train_batch_size).prefetch(tf.data.AUTOTUNE)
        
        self.fit(
            train_dataset,
            epochs = epochs,
            verbose = 0,
            callbacks = self.callbacks,
            batch_size = self.train_batch_size,
            shuffle = shuffle
        )
        
            
    def apply_accumulated_gradients(self):
        """
            Given the proper number of steps for the model to accumulate gradients over time, finally applies gradients to update the model weights.
        """

In [46]:
model = FrailtyModelNN(None, None)

model.define_structure()

x = np.random.normal(size = 10)
t = np.random.exponential(size = 10)
delta = np.random.binomial(n = 1, p = 0.5, size = 10)

model.train_model(x, t, delta,
                  epochs = 10, shuffle = True,
                  optimizer = optimizers.Adam(learning_rate = 0.001),
                  train_batch_size = None, val_batch_size = None,
                  buffer_size = 4096, gradient_accumulation_steps = None,
                  run_eagerly = True, verbose = 1)

INCLUDING NN PARAMETERS IN THE LIST...
VARIABLES MAPPING TO INDEX
{'raw_alpha': 0, 'raw_gamma': 1, 'raw_phi1': 2, 'raw_phi2': 3, 'raw_theta': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>, 'raw_tau': <tf.Tensor: shape=(5,), dtype=int32, numpy=array([1, 2, 3, 4, 5], dtype=int32)>}


0epoch [00:00, ?epoch/s]

{'alpha': <Variable path=frailty_model_nn_20/raw_alpha, shape=(), dtype=float32, value=0.0>, 'gamma': <Variable path=frailty_model_nn_20/raw_gamma, shape=(), dtype=float32, value=0.0>, 'phi1': <Variable path=frailty_model_nn_20/raw_phi1, shape=(), dtype=float32, value=0.0>, 'phi2': <Variable path=frailty_model_nn_20/raw_phi2, shape=(), dtype=float32, value=0.0>}


KeyError: 'raw_alpha'

In [87]:
model.trainable_variables[0].path.split("/")[-1]

'raw_alpha'

In [53]:
model.predict(x[:,None])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


array([[-1.7739912 ],
       [-0.86790764],
       [-0.7947914 ],
       [ 0.41015804],
       [ 0.97464114],
       [ 0.04176952],
       [ 1.498434  ],
       [-0.72059375],
       [-0.85633576],
       [ 0.35985187]], dtype=float32)